# Response Validation Demonstration - LLMManager

This notebook demonstrates the **response validation functionality** of the LLMManager's `converse()` function. Response validation allows you to automatically validate AI responses and retry with different models/regions when validation fails.

## Key Features Demonstrated

- 🔍 **Custom Validation Functions**: Define your own response validation logic
- 🔄 **Automatic Retry on Validation Failure**: Retry with different models when validation fails
- 📊 **Validation Metrics**: Track validation attempts and errors
- 🎯 **Multiple Validation Scenarios**: Content length, format, keywords, JSON structure
- ⚡ **Validation Retry Configuration**: Control retry behavior and delays
- 🔧 **Error Handling**: Comprehensive validation error tracking

## Use Cases

Response validation is particularly useful for:
- Ensuring structured outputs (JSON, XML, specific formats)
- Validating content quality and completeness
- Checking for required keywords or phrases
- Enforcing response length constraints
- Quality assurance in automated workflows

## Setup and Imports

In [ ]:
import sys
import json
import re
import logging
from pathlib import Path
from typing import Dict, Any, List
from datetime import datetime

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import the LLMManager and related classes
from LLMManager import LLMManager
from bedrock.models.llm_manager_structures import (
    AuthConfig, RetryConfig, AuthenticationType, RetryStrategy,
    ResponseValidationConfig, ValidationResult, BedrockResponse
)
from bedrock.exceptions.llm_manager_exceptions import (
    LLMManagerError, ConfigurationError, RetryExhaustedError
)

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")

## Helper Functions

Let's define utility functions for creating validation functions and displaying results.

In [ ]:
def display_response_with_validation(response: BedrockResponse, title: str = "Response"):
    """
    Display a formatted response with validation information.
    """
    print(f"\n{title}")
    print("=" * len(title))
    
    if response.success:
        print(f"✅ Success: {response.success}")
        print(f"🤖 Model: {response.model_used}")
        print(f"🌍 Region: {response.region_used}")
        print(f"🔗 Access Method: {response.access_method_used}")
        print(f"⏱️  Total Duration: {response.total_duration_ms:.2f}ms")
        if response.api_latency_ms:
            print(f"⚡ API Latency: {response.api_latency_ms:.2f}ms")
        
        # Display validation metrics
        validation_metrics = response.get_validation_metrics()
        if validation_metrics:
            print(f"\n🔍 Validation Metrics:")
            print(f"   Validation Attempts: {validation_metrics['validation_attempts']}")
            print(f"   Validation Errors: {validation_metrics['validation_errors']}")
            print(f"   Had Validation Failures: {validation_metrics['had_validation_failures']}")
            if 'successful_validation_attempt' in validation_metrics:
                print(f"   Successful Validation Attempt: {validation_metrics['successful_validation_attempt']}")
        
        # Display content
        content = response.get_content()
        if content:
            print(f"\n💬 Response Content:")
            print("-" * 20)
            print(content)
        
        # Display usage statistics if available
        usage = response.get_usage()
        if usage:
            print(f"\n📊 Token Usage:")
            for key, value in usage.items():
                if value > 0:  # Only show non-zero values
                    print(f"   {key}: {value}")
    else:
        print(f"❌ Success: {response.success}")
        print(f"🔄 Attempts: {len(response.attempts)}")
        if response.get_last_error():
            print(f"❌ Last Error: {response.get_last_error()}")
    
    # Display validation failures if any
    if response.had_validation_failures():
        print(f"\n⚠️  Validation Failures:")
        for i, error in enumerate(response.get_validation_errors(), 1):
            print(f"   {i}. {error.get('error_message', 'Unknown validation error')}")
            if error.get('error_details'):
                print(f"      Details: {error['error_details']}")
    
    if response.warnings:
        print(f"\n⚠️  Warnings:")
        for warning in response.warnings:
            print(f"   - {warning}")


def create_length_validator(min_length: int, max_length: int = None):
    """
    Create a validation function that checks response content length.
    """
    def validate_length(response: BedrockResponse) -> ValidationResult:
        content = response.get_content()
        if not content:
            return ValidationResult(
                success=False,
                error_message="Response has no content",
                error_details={"content_length": 0}
            )
        
        content_length = len(content)
        
        if content_length < min_length:
            return ValidationResult(
                success=False,
                error_message=f"Response too short: {content_length} < {min_length} characters",
                error_details={"content_length": content_length, "min_required": min_length}
            )
        
        if max_length and content_length > max_length:
            return ValidationResult(
                success=False,
                error_message=f"Response too long: {content_length} > {max_length} characters",
                error_details={"content_length": content_length, "max_allowed": max_length}
            )
        
        return ValidationResult(
            success=True,
            error_details={"content_length": content_length}
        )
    
    return validate_length


def create_keyword_validator(required_keywords: List[str], case_sensitive: bool = False):
    """
    Create a validation function that checks for required keywords.
    """
    def validate_keywords(response: BedrockResponse) -> ValidationResult:
        content = response.get_content()
        if not content:
            return ValidationResult(
                success=False,
                error_message="Response has no content to validate"
            )
        
        search_content = content if case_sensitive else content.lower()
        missing_keywords = []
        
        for keyword in required_keywords:
            search_keyword = keyword if case_sensitive else keyword.lower()
            if search_keyword not in search_content:
                missing_keywords.append(keyword)
        
        if missing_keywords:
            return ValidationResult(
                success=False,
                error_message=f"Missing required keywords: {', '.join(missing_keywords)}",
                error_details={
                    "missing_keywords": missing_keywords,
                    "required_keywords": required_keywords
                }
            )
        
        return ValidationResult(
            success=True,
            error_details={"found_keywords": required_keywords}
        )
    
    return validate_keywords


def create_json_validator(required_fields: List[str] = None):
    """
    Create a validation function that ensures response is valid JSON.
    """
    def validate_json(response: BedrockResponse) -> ValidationResult:
        content = response.get_content()
        if not content:
            return ValidationResult(
                success=False,
                error_message="Response has no content to validate as JSON"
            )
        
        # Try to extract JSON from response (handle markdown code blocks)
        json_content = content.strip()
        
        # Remove markdown code block markers if present
        if json_content.startswith('```json'):
            json_content = json_content[7:].strip()
        elif json_content.startswith('```'):
            json_content = json_content[3:].strip()
        
        if json_content.endswith('```'):
            json_content = json_content[:-3].strip()
        
        try:
            parsed_json = json.loads(json_content)
        except json.JSONDecodeError as e:
            return ValidationResult(
                success=False,
                error_message=f"Invalid JSON format: {str(e)}",
                error_details={"json_error": str(e), "content_preview": content[:200]}
            )
        
        # Check for required fields if specified
        if required_fields and isinstance(parsed_json, dict):
            missing_fields = []
            for field in required_fields:
                if field not in parsed_json:
                    missing_fields.append(field)
            
            if missing_fields:
                return ValidationResult(
                    success=False,
                    error_message=f"Missing required JSON fields: {', '.join(missing_fields)}",
                    error_details={
                        "missing_fields": missing_fields,
                        "available_fields": list(parsed_json.keys())
                    }
                )
        
        return ValidationResult(
            success=True,
            error_details={"json_type": type(parsed_json).__name__, "parsed_successfully": True}
        )
    
    return validate_json


print("✅ Helper functions defined!")
print("🔧 Validation functions ready for use")

## Initialize the LLMManager

We'll create an LLMManager instance with multiple models to demonstrate validation retry behavior.

In [ ]:
print("🚀 Initializing LLMManager for Validation Demo...")

# Define models to use (mix of different capabilities)
models = [
    "Claude 3.5 Sonnet v2",  # High-quality responses
    "Claude 3 Haiku",       # Fast and efficient
    "Nova Pro",             # Amazon's model
    "Nova Lite"             # Backup option
]

# Define regions for failover
regions = [
    "us-east-1",
    "us-west-2",
    "eu-west-1"
]

# Configure authentication
auth_config = AuthConfig(
    auth_type=AuthenticationType.PROFILE,
    profile_name="default"
)

# Configure retry behavior for validation demos
retry_config = RetryConfig(
    max_retries=3,
    retry_strategy=RetryStrategy.REGION_FIRST,  # Try different regions first
    retry_delay=1.0,  # 1 second delay between retries
    enable_feature_fallback=True
)

try:
    # Initialize the LLMManager
    manager = LLMManager(
        models=models,
        regions=regions,
        auth_config=auth_config,
        retry_config=retry_config,
        timeout=45  # Longer timeout for validation demos
    )
    
    print(f"✅ LLMManager initialized successfully!")
    print(f"   🤖 Models: {len(manager.get_available_models())}")
    print(f"   🌍 Regions: {len(manager.get_available_regions())}")
    print(f"\n{manager}")
    
    # Validate configuration
    validation = manager.validate_configuration()
    print(f"\n🔍 Configuration Validation:")
    print(f"   Valid: {'✅' if validation['valid'] else '❌'} {validation['valid']}")
    print(f"   Auth Status: {validation['auth_status']}")
    print(f"   Model-Region Combinations: {validation['model_region_combinations']}")
    
    if validation['warnings']:
        print(f"   ⚠️  Warnings: {len(validation['warnings'])}")
    if validation['errors']:
        print(f"   ❌ Errors: {len(validation['errors'])}")
        for error in validation['errors']:
            print(f"      - {error}")

except Exception as e:
    print(f"❌ Error initializing LLMManager: {e}")
    print("\n💡 Troubleshooting tips:")
    print("   1. Ensure AWS credentials are configured")
    print("   2. Check that you have access to the specified models and regions")
    print("   3. Verify network connectivity to AWS")
    raise

## Example 1: Basic Response Length Validation 📏

Let's start with a simple validation that ensures responses meet minimum length requirements.

In [ ]:
print("📏 Example 1: Response Length Validation")
print("=" * 40)

# Create a length validator that requires at least 100 characters
length_validator = create_length_validator(min_length=100, max_length=500)

# Create validation configuration
validation_config = ResponseValidationConfig(
    response_validation_function=length_validator,
    response_validation_retries=2,  # Try validation 2 times per model
    response_validation_delay=0.5   # 0.5 second delay between validation retries
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Please provide a brief explanation of quantum computing. Keep it concise but informative."
            }
        ]
    }
]

try:
    print("🔍 Sending request with length validation (min: 100, max: 500 characters)...")
    
    response = manager.converse(
        messages=messages,
        inference_config={
            "maxTokens": 200,  # Limit tokens to potentially trigger validation failures
            "temperature": 0.7
        },
        response_validation_config=validation_config
    )
    
    display_response_with_validation(response, "📏 Length Validation Result")

except RetryExhaustedError as e:
    print(f"❌ All validation attempts failed: {e}")
    print(f"   Models tried: {e.models_tried}")
    print(f"   Regions tried: {e.regions_tried}")
except Exception as e:
    print(f"❌ Error in length validation demo: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 2: Keyword Validation 🔍

Demonstrate validation that checks for required keywords in the response.

In [ ]:
print("🔍 Example 2: Keyword Validation")
print("=" * 32)

# Create a keyword validator that requires specific terms
required_keywords = ["advantages", "disadvantages", "applications"]
keyword_validator = create_keyword_validator(
    required_keywords=required_keywords,
    case_sensitive=False
)

# Create validation configuration with more retries
validation_config = ResponseValidationConfig(
    response_validation_function=keyword_validator,
    response_validation_retries=3,
    response_validation_delay=0.0  # No delay for faster demo
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Explain artificial intelligence. Make sure to cover both the positive and negative aspects, and mention where it's commonly used."
            }
        ]
    }
]

try:
    print(f"🔍 Sending request with keyword validation...")
    print(f"   Required keywords: {', '.join(required_keywords)}")
    
    response = manager.converse(
        messages=messages,
        inference_config={
            "maxTokens": 400,
            "temperature": 0.5
        },
        response_validation_config=validation_config
    )
    
    display_response_with_validation(response, "🔍 Keyword Validation Result")

except RetryExhaustedError as e:
    print(f"❌ All validation attempts failed: {e}")
    print(f"   This may indicate that none of the models provided all required keywords")
except Exception as e:
    print(f"❌ Error in keyword validation demo: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 3: JSON Structure Validation 🧮

Demonstrate validation for structured JSON responses - very useful for API integrations.

In [ ]:
print("🧮 Example 3: JSON Structure Validation")
print("=" * 37)

# Create a JSON validator that requires specific fields
required_json_fields = ["title", "summary", "pros", "cons"]
json_validator = create_json_validator(required_fields=required_json_fields)

# Create validation configuration
validation_config = ResponseValidationConfig(
    response_validation_function=json_validator,
    response_validation_retries=2,
    response_validation_delay=0.0
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Please provide information about electric vehicles in JSON format with the following structure: {\"title\": \"Brief title\", \"summary\": \"Overview summary\", \"pros\": [\"list\", \"of\", \"advantages\"], \"cons\": [\"list\", \"of\", \"disadvantages\"]}. Respond only with valid JSON, no additional text or markdown formatting."
            }
        ]
    }
]

try:
    print(f"🧮 Sending request with JSON validation...")
    print(f"   Required JSON fields: {', '.join(required_json_fields)}")
    
    response = manager.converse(
        messages=messages,
        inference_config={
            "maxTokens": 600,
            "temperature": 0.3  # Lower temperature for more structured output
        },
        response_validation_config=validation_config
    )
    
    display_response_with_validation(response, "🧮 JSON Validation Result")
    
    # Parse and display the JSON if successful
    if response.success:
        try:
            content = response.get_content()
            # Handle potential markdown formatting
            json_content = content.strip()
            if json_content.startswith('```json'):
                json_content = json_content[7:].strip()
            elif json_content.startswith('```'):
                json_content = json_content[3:].strip()
            if json_content.endswith('```'):
                json_content = json_content[:-3].strip()
            
            parsed_data = json.loads(json_content)
            print(f"\n✅ Successfully parsed JSON:")
            print(json.dumps(parsed_data, indent=2))
        except json.JSONDecodeError as e:
            print(f"\n⚠️  Failed to parse JSON for display: {e}")

except RetryExhaustedError as e:
    print(f"❌ All validation attempts failed: {e}")
    print(f"   This may indicate that none of the models provided valid JSON")
except Exception as e:
    print(f"❌ Error in JSON validation demo: {e}")
    print(f"   Type: {type(e).__name__}")